In [1]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import libarys
import sqlite3
import pandas as pd
import zipfile

In [3]:
# download dataset form Kaggle
!kaggle datasets download -d heptapod/titanic

Dataset URL: https://www.kaggle.com/datasets/heptapod/titanic
License(s): DbCL-1.0
titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
# extracting the dataset
with zipfile.ZipFile("titanic.zip", 'r') as zip_ref:
    zip_ref.extractall("titanic_data")

In [5]:
# load the dataset
df = pd.read_csv("titanic_data_train.csv")

In [6]:
# display the first few rows of the dataset
print("Dataset Preview:")
print(df.head())

Dataset Preview:
   Passengerid   Age     Fare  Sex  sibsp  zero  zero.1  zero.2  zero.3  \
0            1  22.0   7.2500    0      1     0       0       0       0   
1            2  38.0  71.2833    1      1     0       0       0       0   
2            3  26.0   7.9250    1      0     0       0       0       0   
3            4  35.0  53.1000    1      1     0       0       0       0   
4            5  35.0   8.0500    0      0     0       0       0       0   

   zero.4  ...  zero.12  zero.13  zero.14  Pclass  zero.15  zero.16  Embarked  \
0       0  ...        0        0        0       3        0        0       2.0   
1       0  ...        0        0        0       1        0        0       0.0   
2       0  ...        0        0        0       3        0        0       2.0   
3       0  ...        0        0        0       1        0        0       2.0   
4       0  ...        0        0        0       3        0        0       2.0   

   zero.17  zero.18  2urvived  
0        0   

In [7]:
# create a SQLite database and load the DataFrame into it
conn = sqlite3.connect('titanic.db')
df.to_sql('titanic', conn, if_exists='replace', index=False)

1309

In [8]:
print(df.columns)

Index(['Passengerid', 'Age', 'Fare', 'Sex', 'sibsp', 'zero', 'zero.1',
       'zero.2', 'zero.3', 'zero.4', 'zero.5', 'zero.6', 'Parch', 'zero.7',
       'zero.8', 'zero.9', 'zero.10', 'zero.11', 'zero.12', 'zero.13',
       'zero.14', 'Pclass', 'zero.15', 'zero.16', 'Embarked', 'zero.17',
       'zero.18', '2urvived'],
      dtype='object')


In [9]:
# rename the problematic column
df.rename(columns={"2urvived": "Survived"}, inplace=True)

# verify the column names
print(df.columns)

Index(['Passengerid', 'Age', 'Fare', 'Sex', 'sibsp', 'zero', 'zero.1',
       'zero.2', 'zero.3', 'zero.4', 'zero.5', 'zero.6', 'Parch', 'zero.7',
       'zero.8', 'zero.9', 'zero.10', 'zero.11', 'zero.12', 'zero.13',
       'zero.14', 'Pclass', 'zero.15', 'zero.16', 'Embarked', 'zero.17',
       'zero.18', 'Survived'],
      dtype='object')


In [10]:
# reload the updated DataFrame into SQLite
df.to_sql('titanic', conn, if_exists='replace', index=False)

1309

In [11]:
# SQL Query
print("\nQuery 1: Count the number of passengers in each class")
query_1 = """
SELECT Pclass, COUNT(*) AS PassengerCount
FROM titanic
GROUP BY Pclass
ORDER BY PassengerCount DESC
"""
result_1 = pd.read_sql_query(query_1, conn)
print(result_1)


Query 1: Count the number of passengers in each class
   Pclass  PassengerCount
0       3             709
1       1             323
2       2             277


In [12]:
print("\nQuery 2: Calculate the survival rate for each gender")
query_2 = """
SELECT Sex, AVG(Survived) AS SurvivalRate
FROM titanic
GROUP BY Sex
"""
result_2 = pd.read_sql_query(query_2, conn)
print(result_2)


Query 2: Calculate the survival rate for each gender
   Sex  SurvivalRate
0    0        0.1293
1    1        0.5000


In [13]:
print("\nQuery 3: Find the top 5 passengers with the highest fares")
query_3 = """
SELECT PassengerId, Fare
FROM titanic
ORDER BY Fare DESC
LIMIT 5
"""
result_3 = pd.read_sql_query(query_3, conn)
print(result_3)


Query 3: Find the top 5 passengers with the highest fares
   Passengerid      Fare
0          259  512.3292
1          680  512.3292
2          738  512.3292
3         1235  512.3292
4           28  263.0000


In [14]:
# close the SQLite connection
conn.close()